### Why Use Record Linkage?
Record linkage is used **when two datasets contain records about the same entities (e.g., people, businesses, products) but lack unique identifiers.** Instead of a straightforward merge() (which requires a unique key like an ID), record linkage:

1. **Identifies likely matches** based on shared attributes (e.g., names, addresses, birthdates).
2. **Handles inconsistencies**, such as typos, abbreviations, or missing values.
3. **Links records** that refer to the same entity across both datasets.

Since record linkage is used **when unique identifiers are missing**, a simple merge() won't work. Instead, **fuzzy matching techniques** (like Levenshtein distance or phonetic matching) are used to identify records that are likely the same.

In [98]:
import recordlinkage
from recordlinkage.datasets import load_febrl4
import pandas as pd
dfA, dfB = load_febrl4()

# Create an indexing object - this object helps generate pairs from dfA and dfB
indexer = recordlinkage.Index()

# Group together rows by blocking on the 'state' column
indexer.block('state')

# Generate pairs based on the blocking 
pairs = indexer.index(dfA, dfB)

print(pairs)

MultiIndex([('rec-1070-org', 'rec-2642-dup-0'),
            ('rec-1070-org', 'rec-1424-dup-0'),
            ('rec-1070-org',  'rec-825-dup-0'),
            ('rec-1070-org',  'rec-520-dup-0'),
            ('rec-1070-org',  'rec-147-dup-0'),
            ('rec-1070-org',  'rec-919-dup-0'),
            ('rec-1070-org', 'rec-3156-dup-0'),
            ('rec-1070-org', 'rec-1026-dup-0'),
            ('rec-1070-org', 'rec-4342-dup-0'),
            ('rec-1070-org', 'rec-2871-dup-0'),
            ...
            (  'rec-66-org',  'rec-196-dup-0'),
            (  'rec-66-org',  'rec-546-dup-0'),
            (  'rec-66-org',    'rec-6-dup-0'),
            (  'rec-66-org', 'rec-2062-dup-0'),
            (  'rec-66-org', 'rec-2281-dup-0'),
            (  'rec-66-org', 'rec-4459-dup-0'),
            (  'rec-66-org',  'rec-670-dup-0'),
            (  'rec-66-org', 'rec-3152-dup-0'),
            (  'rec-66-org', 'rec-3363-dup-0'),
            (  'rec-66-org', 'rec-4495-dup-0')],
           names=['rec_

In [99]:
# Create a comparison object
compare_cl = recordlinkage.Compare()

# Exact match on 'given_name'
compare_cl.exact('given_name', 'given_name', label ='first_name')

# Exact match on 'date_of_birth'
compare_cl.exact('date_of_birth', 'date_of_birth', label ='date_of_birth')

# Exact match on 'soc_sec_id'
compare_cl.exact('soc_sec_id', 'soc_sec_id', label ='soc_sec_id')

# String similarity comparison on 'address_1' using Levenshtein distance with a threshold of 0.85
compare_cl.string('address_1', 'address_1', threshold=0.85, label='address_1')

# Compute potential matches using the generated index pairs
potential_matches = compare_cl.compute(pairs, dfA, dfB) 

# Multi-Index Data Frame is computed 

# Columns: A binary indicator (1 or 0) for each field (first_name, date_of_birth, soc_sec_id, address_1).

# If a field matches exactly or meets the similarity threshold, it is marked as 1; otherwise, 0.

# Find matches where the sum of matches across columns is >= 3
# Helps identify strong duplicates, likely the same entity in dfA and dfB
matched_rows = potential_matches[potential_matches.sum(axis=1) >= 3]

print(matched_rows)

                             first_name  date_of_birth  soc_sec_id  address_1
rec_id_1     rec_id_2                                                        
rec-4405-org rec-4405-dup-0           1              1           1        1.0
rec-1288-org rec-1288-dup-0           1              1           1        1.0
rec-3585-org rec-3585-dup-0           1              1           1        1.0
rec-298-org  rec-298-dup-0            1              1           1        0.0
rec-1985-org rec-1985-dup-0           0              1           1        1.0
...                                 ...            ...         ...        ...
rec-3877-org rec-3877-dup-0           1              1           0        1.0
rec-1604-org rec-1604-dup-0           0              1           1        1.0
rec-1003-org rec-1003-dup-0           0              1           1        1.0
rec-4883-org rec-4883-dup-0           1              1           1        1.0
rec-66-org   rec-66-dup-0             1              1          

In [101]:
# Get indices from dfB only
duplicates_rows = matched_rows.index.get_level_values(1)

# Filters dfB to only include the rows that were identified as duplicates in matched_rows
dfB_duplicates = dfB[dfB.index.isin(duplicates_rows)]

# Finding new rows in dfB
dfB_new = dfB[~dfB.index.isin(duplicates_rows)]

# Link the data frames
full_data = pd.concat([dfA, dfB_new], ignore_index=True) 

# full_data contains all unique records from both datasets, with duplicates removed.
print(full_data)

# ensures there are no duplicate rows in the full_data dataframe
print('Number of duplicate rows: ', full_data.duplicated().sum())  

     given_name   surname street_number            address_1  \
0      michaela   neumann             8       stanley street   
1      courtney   painter            12    pinkerton circuit   
2       charles     green            38  salkauskas crescent   
3       vanessa      parr           905       macquoid place   
4       mikayla  malloney            37        randwick road   
...         ...       ...           ...                  ...   
5847     jordan  kondkaov           NaN          fleay place   
5848       kyle      ryna            33                  NaN   
5849    jackson  alderman            84                  NaN   
5850       zali     white             6       mildura street   
5851       saah   beattih            60          kay's place   

               address_2            suburb postcode state date_of_birth  \
0                  miami     winston hills     4223   nsw      19151111   
1             bega flats         richlands     4560   vic      19161214   
2     